In [1]:
import pandas as pd

In [2]:
params_mat = pd.read_csv('plot_data.csv')

In [4]:
name= "data_2real1bigdet"
model_name="ResNet"
stoc=10

In [11]:
(params_mat['data'] == name) & (params_mat['model'] == model_name)

0      False
1      False
2      False
3      False
4       True
       ...  
136    False
137    False
138    False
139    False
140    False
Length: 141, dtype: bool

In [14]:
params_row = params_mat.loc[(params_mat['data'] == name) & (params_mat['model'] == model_name) & (params_mat['stoc'] == stoc )]


In [16]:
params_row['ks']

9    [1, 7, 9]
Name: ks, dtype: object

In [30]:
params_row['nf'].values[0].dtype


dtype('float64')

In [25]:
from ast import literal_eval

In [29]:
literal_eval(params_row['ks'].values[0])[0]

AttributeError: 'int' object has no attribute 'dtype'

In [7]:
from tsai import *
import inspect

'''script containing hyperparameter optimisation functions, model fitting functions and output analysis functions'''
from collections import Counter
import statistics
import timeit
import itertools
import warnings
from tsai.all import *

import numpy as np
import torch
import torch.nn as nn

import optuna
from optuna.samplers import TPESampler
import sklearn.metrics as skm
import pandas as pd


from sklearn.model_selection import StratifiedKFold
from torch.utils.data import WeightedRandomSampler

from sklearn.utils.class_weight import compute_class_weight
from optuna.integration import FastAIPruningCallback
from fastai.vision.all import *
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import Data_load_neat as Data_load
import explr_inter


In [12]:
inspect.signature(ts_learner)

<Signature (dls, arch=None, c_in=None, c_out=None, seq_len=None, d=None, splitter=<function trainable_params at 0x7f454b9f6ee0>, loss_func=None, opt_func=<function Adam at 0x7f454b8cf940>, lr=0.001, cbs=None, metrics=None, path=None, model_dir='models', wd=None, wd_bn_bias=False, train_bn=True, moms=(0.95, 0.85, 0.95), train_metrics=False, valid_metrics=True, seed=None, *, device=None, verbose=False, pretrained=False, weights_path=None, exclude_head=True, cut=-1, init=None, arch_config={}) -> fastai.learner.Learner>

In [36]:
inspect.signature(TSDatasets)

<Signature (items: 'list' = None, tfms: 'MutableSequence | Pipeline' = None, tls: 'TfmdLists' = None, n_inp: 'int' = None, dl_type=None, *, use_list: 'bool' = None, do_setup: 'bool' = True, split_idx: 'int' = None, train_setup: 'bool' = True, splits: 'list' = None, types=None, verbose: 'bool' = False)>

In [18]:
filepath="/home/DIDE/smishra/Simulations/"
randnum_split = 7
randnum_train=9
name="data_2real1newerbigdet"
model_name="LSTMAttention"
subset=-1

np.random.seed(randnum_split)  # Numpy vars
torch.manual_seed(randnum_train)  # PyTorch vars
random.seed(randnum_split)  # Python
torch.cuda.manual_seed(randnum_train)
torch.cuda.manual_seed_all(randnum_train)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Function to load in data
#X_raw, y_raw = Data_load.load_data(name=name,filepath=filepath,subset=subset)


## function to load all the data from the filepath

X_raw = np.load("".join([filepath,"input_data/",name, "_X.npy"])).astype(np.float32)

Y_raw = np.squeeze(np.load("".join([filepath,"input_data/",name, "_YH.npy"])))
y_raw = Y_raw[:, np.shape(Y_raw)[1] - 1]

### Helen messing around
if subset>0:
    X_raw=X_raw[:subset,:,:]
    y_raw=y_raw[:subset]
## Helen stop messing around

## Function to obtain the train/test split
#X_trainvalid, Y_trainvalid, X_test, Y_test, splits = Data_load.split_data(X=X_raw,Y=y_raw,randnum=randnum_split)



## split out the test set
splits = get_splits(
        y_raw,
        valid_size=0.2,
        stratify=True,
        shuffle=True,
        test_size=0,
        show_plot=False,
        random_state=randnum_split
        )
X_trainvalid, X_test = X_raw[splits[0]], X_raw[splits[1]]
Y_trainvalid, Y_test = y_raw[splits[0]], y_raw[splits[1]]

print(f' shape = {splits[0].shape}')

print(f'sum = {np.sum(splits[0])}; mean = {np.mean(splits[0])}; var = {np.var(splits[0])}')

AttributeError: 'list' object has no attribute 'shape'

In [31]:
splits[0]
stoc=0.1

In [32]:
sum(splits[0]) / len(splits[0]) 

49978.4667625

In [33]:
num1s=np.sum(Y_trainvalid)
num10=math.ceil(stoc*num1s)
which1=np.where(Y_trainvalid==1)[0]
which0=np.where(Y_trainvalid==0)[0]

# Randomly selects the correct number of 1s/0s that will be switched to 0s/1s
which10=random.sample(list(which1),num10)
which01=random.sample(list(which0),num10)

In [35]:
np.mean(which10)


38761.38983050847